# MLiA Ch. 6: support vector machines

## SVM classifier 수식화 

원래의 문제는 N개의 선형 부등식(각 training data가 hyperplane과의 거리는 (margin-slack)보다 큼)을 조건으로 가진 2차 함수(margin 크기 역수의 제곱)의 최소화하는 classifier($W$, $b$)로서, KKT(Karush-Kuhn-Tucker) 조건과 convex optimization 문제에 대한 Wolfe dual 표현 이용하여 다음을 최대화하는 $\alpha_i$(Lagrange multiplier)를 구하는 문제로 바꾸어 해결한다. 

$$ \sum^N_{i=1}\alpha_i t_i=0 $$
$$ 0\leq\alpha_i\leq C, i=1, \dots , N $$
$$ \tilde{L}(\alpha)=\sum^N_{i=1}\alpha_i - \frac{1}{2}\sum^N_{i=1}\sum^N_{j=1}\alpha_i\alpha_j t_i t_j \mathbf{x}_i^T \mathbf{x}_j$$

- 참고
  - 오일석, "패턴인식," 교보문고, 2008
  - 한학용, "패턴인식 개론," 한빛미디어, 2009
  - S. Theodoridis and K. Koutroumbas, "Pattern Recognition," Elsevier, 2009

![](wolfe_dual_func.png)


## SMO 알고리즘 이용한 SVM 학습 

여러 가지 변환을 통해 학습을 좀더 쉬운 문제로 바꾸었으나 수많은 2차 항(약 $\frac{N^2}{2}$개)이 포함된 식을 최적화해야 하는 여전히 어려운 문제로서 SMO, Cutting-plane 등의 SVM 학습 알고리즘들이 있다. 

- SMO(Sequential Minimal Optimization): 1996년 John Platt 발표한 SVM 학습 알고리즘으로 전체 문제를 작은 문제들로 나누어 순차적으로 풀어 나감

### Simplified SMO

SMO를 단순화한 version부터 시작한다.

In [57]:
# Helper functions

from numpy import *
from time import sleep

def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

Training set 파일에서 입력 data vector와 해당 class를 각각 load

In [58]:
dataArr,labelArr = loadDataSet('testSet.txt')

In [59]:
dataArr

[[3.542485, 1.977398],
 [3.018896, 2.556416],
 [7.55151, -1.58003],
 [2.114999, -0.004466],
 [8.127113, 1.274372],
 [7.108772, -0.986906],
 [8.610639, 2.046708],
 [2.326297, 0.265213],
 [3.634009, 1.730537],
 [0.341367, -0.894998],
 [3.125951, 0.293251],
 [2.123252, -0.783563],
 [0.887835, -2.797792],
 [7.139979, -2.329896],
 [1.696414, -1.212496],
 [8.117032, 0.623493],
 [8.497162, -0.266649],
 [4.658191, 3.507396],
 [8.197181, 1.545132],
 [1.208047, 0.2131],
 [1.928486, -0.32187],
 [2.175808, -0.014527],
 [7.886608, 0.461755],
 [3.223038, -0.552392],
 [3.628502, 2.190585],
 [7.40786, -0.121961],
 [7.286357, 0.251077],
 [2.301095, -0.533988],
 [-0.232542, -0.54769],
 [3.457096, -0.082216],
 [3.023938, -0.057392],
 [8.015003, 0.885325],
 [8.991748, 0.923154],
 [7.916831, -1.781735],
 [7.616862, -0.217958],
 [2.450939, 0.744967],
 [7.270337, -2.507834],
 [1.749721, -0.961902],
 [1.803111, -0.176349],
 [8.804461, 3.044301],
 [1.231257, -0.568573],
 [2.074915, 1.41055],
 [-0.743036, -1.73

In [60]:
shape(dataArr)

(100, 2)

In [61]:
shape(labelArr)

(100,)

In [62]:
dataArrMat = mat(dataArr)
shape(dataArrMat)

(100, 2)

In [63]:
labelArrMat = mat(labelArr)
shape(labelArrMat.transpose())

(100, 1)

In [64]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    b = 0; m,n = shape(dataMatrix)    # m: training data vector 개수, n: training data vector의 차원 수(feature 개수)
    alphas = mat(zeros((m,1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        # Training data vector set에 대해 순차 수행
        for i in range(m):          
            # fxi = i번째 data vector에 대한 현재까지 훈련된 classifier 적용 결과
            # W = multiply(alphas,labelMat).T*(dataMatrix) 
            # multiply(): element-wise multiplication (NOT matrix multiplication), '*': matrix multiplication
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b   
            # Ei = i번째 data vector에 대한 error(예측치-정답)
            Ei = fXi - float(labelMat[i])#if checks if an example violates KKT conditions
            # KKT 조건을 만족하면서 
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print "L==H"; continue
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0: print "eta>=0"; continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])#update i by the same amount as j
                                                                        #the update is in the oppostie direction
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print "iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print "iteration number: %d" % iter
    return b,alphas

In [65]:
b,alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

iter: 0 i:0, pairs changed 1
L==H
L==H
L==H
iter: 0 i:23, pairs changed 2
j not moving enough
L==H
j not moving enough
j not moving enough
iter: 0 i:54, pairs changed 3
L==H
j not moving enough
L==H
L==H
j not moving enough
iter: 0 i:94, pairs changed 4
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
L==H
L==H
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
iter: 1 i:97, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
L

In [66]:
b

matrix([[-3.85837696]])

In [67]:
alphas[alphas>0]

matrix([[ 0.1172426 ,  0.24724674,  0.36448934]])

In [68]:
for i in range(100):
    if alphas[i]>0.0: print dataArr[i],labelArr[i]

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[6.080573, 0.418886] 1.0


In [69]:
shape(alphas[alphas>0])

(1, 3)

In [72]:
%matplotlib inline
import plotSupportVectors

### Full SMO